Silver Layer 🥈

Imports

In [75]:
import pandas as pd 
import numpy as np 
import string
import datetime

Load the Bronze layer data

In [76]:
df=pd.read_parquet('./Bronze layer/Bronze.parquet')

Generalized Data Validation and Cleaning functions

In [77]:
def drop_null_columns(df, threshold=0.2):
    '''dropping columns with null values more than the threshold value'''
    null_percentage = df.isnull().mean()
    columns_to_keep= null_percentage[null_percentage<threshold].index 
    
    if len(columns_to_keep)==len(df.columns):
        print("No columns removed")
    else:
        print(f"Removing: {[c for c in df.columns if c not in columns_to_keep]}")
    return df[columns_to_keep]

In [78]:
def fill_missing_values(df,columns_defaults:dict):
    '''filling missing values in the columns'''
    for column,default_value in columns_defaults.items():
        df[column] = df[column].fillna(default_value)
    return df

In [79]:
def convert_column_types(df,column_types:dict):
    '''converting the columns to the desired datatype'''
    try:
        for column,dtype in column_types.items():
               df[column] = df[column].astype(dtype)
        return df
    except Exception as e:
         print(f"{column} caused an issue")
         raise e

In [80]:
def remove_punctuation(df, columns:list):
    '''removing punctuations from the columns'''
    for c in columns:
        df.loc[:,c] = df[c].str.replace(r'[^\w\s]|_','',regex=True)
    return df

In [81]:
def check_formats(df, expected_formats: dict):
    '''checking formats of the columns'''
    incorrect_formats =[]
    for column,datatype in df.dtypes.to_dict().items():
        expected_type = expected_formats.get(column)
        if expected_type != datatype:
            incorrect_formats.append((column,datatype,expected_type))
    
    incorrect_columns =[c[0] for c in incorrect_formats]
    correct_format_count = len([c for c in df.columns if c not in incorrect_columns])
    if incorrect_formats:
        print('Below are incorrect formats')
        print('_'*50)
        print(f"Correct column count: {correct_format_count}")
        return pd.DataFrame(incorrect_formats, columns=['Columns','Actual','Expected'])
    else:
        print("Validation Complete, no discrepancies")

In [82]:
def check_similarity(word1:str,word2:str)->float:
    '''checking similarity between two words'''
    word_set1=set(word1)
    word_set2=set(word2)

    intersection = word_set1.intersection(word_set2)
    intersection_count=len(intersection)

    total_char_count = len(word_set1.union(word_set2))

    similarity=intersection_count/total_char_count
    return similarity

In [83]:
def check_mispelling(dataframe: pd.DataFrame, column: str, similarity_threshold: float)->pd.DataFrame:
    '''checking for mispellings of words'''
    all_unique_values = list(set(dataframe[column].tolist()))

    similarity_list=[]

    for n in range(len(all_unique_values)):
        value1=all_unique_values[n]
        for n2 in range(n+1,len(all_unique_values)):
            value2=all_unique_values[n2]
            similarity= round(check_similarity(value1,value2),4)
            if similarity >=similarity_threshold:
                similarity_list.append([value1,value2,similarity])
    return pd.DataFrame(similarity_list,columns=['name1','name2','similarity'])

Apply Validations and Cleaning

In [84]:
df.head()

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,...,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,filename,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,...,0.0,Yes,No,1-Up Mushroom,64,26,4.26,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,...,4.0,No,No,"Red Shell, Super Star",335,40,5.00,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
2,Waluigi,None,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,...,1.0,No,Yes,Green Shell,182,57,5.50,Mario Kart 8 Deluxe,mario_data_20250101.csv,2025-10-28 15:21:26.079051
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,...,5.0,No,Yes,1-Up Mushroom,333,84,6.00,Super Mario Bros.,mario_data_20250101.csv,2025-10-28 15:21:26.079051
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,...,2.0,Yes,No,"Red Shell, Banana Peel, Fire Flower",461,55,7.00,Super Mario World,mario_data_20250101.csv,2025-10-28 15:21:26.079051


In [ ]:
# checking the number of null values
df.isnull().sum() 


Player Name                      9040
Team                             2960
World                               0
Vehicle Type                        0
Companion                           0
Kart Racing Rank                 9280
Platforming Rank                    0
Boss Battle Rank                    0
Power-Ups Used                      0
Kart Role                           0
Team Points                         0
Lives Lost                          0
Participation in Battle Mode        0
Mushroom Cup Participation      12400
Power-Ups Owned                     0
Coins Spent in Toad Town            0
Levels Completed                    0
Times Hit by Enemies                0
Primary Game                        0
filename                            0
loadDateTimeStamp                   0
dtype: int64

In [ ]:
# checking percentage of null values within each column
df.isnull().mean()*100

Player Name                     18.833333
Team                             6.166667
World                            0.000000
Vehicle Type                     0.000000
Companion                        0.000000
Kart Racing Rank                19.333333
Platforming Rank                 0.000000
Boss Battle Rank                 0.000000
Power-Ups Used                   0.000000
Kart Role                        0.000000
Team Points                      0.000000
Lives Lost                       0.000000
Participation in Battle Mode     0.000000
Mushroom Cup Participation      25.833333
Power-Ups Owned                  0.000000
Coins Spent in Toad Town         0.000000
Levels Completed                 0.000000
Times Hit by Enemies             0.000000
Primary Game                     0.000000
filename                         0.000000
loadDateTimeStamp                0.000000
dtype: float64

In [ ]:
# checking the columns that have null values greater than threshold
drop_null_columns(df,threshold=0.2)

Removing: ['Mushroom Cup Participation']


,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,filename,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,-34.0,0.000000,Yes,1-Up Mushroom,64,26,4.26,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,149.0,4.000000,No,"Red Shell, Super Star",335,40,5.00,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
2,Waluigi,None,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,174.0,1.000000,No,Green Shell,182,57,5.50,Mario Kart 8 Deluxe,mario_data_20250101.csv,2025-10-28 15:21:26.079051
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,-1.0,5.000000,No,1-Up Mushroom,333,84,6.00,Super Mario Bros.,mario_data_20250101.csv,2025-10-28 15:21:26.079051
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,28.0,2.000000,Yes,"Red Shell, Banana Peel, Fire Flower",461,55,7.00,Super Mario World,mario_data_20250101.csv,2025-10-28 15:21:26.079051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,Toadette,Green Caps,Star World,Mach 8,Luma,B,A,S,8,Drifter,-84.0,0.000000,No,"1-Up Mushroom, Super Mushroom",172,43,5.00,Super Mario Bros.,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47996,Bowser Jr.,Dino Buddies,Donut Plains,Circuit Special,Koopa Troopa,C,C,B,13,Speedster,110.0,1.083565,No,Fire Flower,11,72,9.00,Super Mario World,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47997,Luigi,Green Caps,dONUT pLAINS,Biddybuggy,Shy Guy,A,D,D,27,Drifter,-38.0,4.000000,Yes,Super Star,460,25,2.00,Super Mario Odyssey,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47998,Toadette,Koopa Clan,mUSHROOM kINGDOM,Circuit Special,Yoshi,B,A,B,8,Item Specialist,111.0,0.000000,Yes,"Super Star, Banana Peel, Green Shell",82,39,4.00,Mario Kart 8 Deluxe,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040


In [ ]:
# checking the data
df.head()

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,...,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,filename,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,...,0.0,Yes,No,1-Up Mushroom,64,26,4.26,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,...,4.0,No,No,"Red Shell, Super Star",335,40,5.00,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
2,Waluigi,None,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,...,1.0,No,Yes,Green Shell,182,57,5.50,Mario Kart 8 Deluxe,mario_data_20250101.csv,2025-10-28 15:21:26.079051
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,...,5.0,No,Yes,1-Up Mushroom,333,84,6.00,Super Mario Bros.,mario_data_20250101.csv,2025-10-28 15:21:26.079051
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,...,2.0,Yes,No,"Red Shell, Banana Peel, Fire Flower",461,55,7.00,Super Mario World,mario_data_20250101.csv,2025-10-28 15:21:26.079051


In [ ]:
# checking the minimum value of power-ups used to assign a missing value in next step
min(df['Power-Ups Used'])

5

In [ ]:
# filling null values in each column with a default value
fill_missing_values(df,{'Player Name':'Unknown Player', 'Team':'Unknown Team','World':'Unknown World','Vehicle Type': 'Unknown Vehicle','Companion':'Unkown Companion','Kart Racing Rank':'Unkown Rank','Platforming Rank':'Unkown Rank','Boss Battle Rank':'Unkown Rank','Power-Ups Used':0,'Kart Role':'Unkown Role','Team Points':0.0,'Lives Lost':0.0,'Participation in Battle Mode':'Unkown','Mushroom Cup Participation':'Unkown','Power-Ups Owned':'None','Coins Spent in Toad Town':0,'Levels Completed':0,'Times Hit by Enemies':0.0,'Primary Game':'N/A'})

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,...,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,filename,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,...,0.000000,Yes,No,1-Up Mushroom,64,26,4.26,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,...,4.000000,No,No,"Red Shell, Super Star",335,40,5.00,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
2,Waluigi,Unknown Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,...,1.000000,No,Yes,Green Shell,182,57,5.50,Mario Kart 8 Deluxe,mario_data_20250101.csv,2025-10-28 15:21:26.079051
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,...,5.000000,No,Yes,1-Up Mushroom,333,84,6.00,Super Mario Bros.,mario_data_20250101.csv,2025-10-28 15:21:26.079051
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,...,2.000000,Yes,No,"Red Shell, Banana Peel, Fire Flower",461,55,7.00,Super Mario World,mario_data_20250101.csv,2025-10-28 15:21:26.079051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,Toadette,Green Caps,Star World,Mach 8,Luma,B,A,S,8,Drifter,...,0.000000,No,Yes,"1-Up Mushroom, Super Mushroom",172,43,5.00,Super Mario Bros.,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47996,Bowser Jr.,Dino Buddies,Donut Plains,Circuit Special,Koopa Troopa,C,C,B,13,Speedster,...,1.083565,No,Unkown,Fire Flower,11,72,9.00,Super Mario World,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47997,Luigi,Green Caps,dONUT pLAINS,Biddybuggy,Shy Guy,A,D,D,27,Drifter,...,4.000000,Yes,Unkown,Super Star,460,25,2.00,Super Mario Odyssey,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47998,Toadette,Koopa Clan,mUSHROOM kINGDOM,Circuit Special,Yoshi,B,A,B,8,Item Specialist,...,0.000000,Yes,Yes,"Super Star, Banana Peel, Green Shell",82,39,4.00,Mario Kart 8 Deluxe,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040


In [ ]:
# validating whether all null values have been filled with a default value
df.isnull().mean()*100

Player Name                     0.0
Team                            0.0
World                           0.0
Vehicle Type                    0.0
Companion                       0.0
Kart Racing Rank                0.0
Platforming Rank                0.0
Boss Battle Rank                0.0
Power-Ups Used                  0.0
Kart Role                       0.0
Team Points                     0.0
Lives Lost                      0.0
Participation in Battle Mode    0.0
Mushroom Cup Participation      0.0
Power-Ups Owned                 0.0
Coins Spent in Toad Town        0.0
Levels Completed                0.0
Times Hit by Enemies            0.0
Primary Game                    0.0
filename                        0.0
loadDateTimeStamp               0.0
dtype: float64

In [ ]:
# removing punctuations from Kart Role
remove_punctuation(df,['Kart Role'])

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,...,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,filename,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,...,0.000000,Yes,No,1-Up Mushroom,64,26,4.26,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,...,4.000000,No,No,"Red Shell, Super Star",335,40,5.00,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
2,Waluigi,Unknown Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,...,1.000000,No,Yes,Green Shell,182,57,5.50,Mario Kart 8 Deluxe,mario_data_20250101.csv,2025-10-28 15:21:26.079051
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,...,5.000000,No,Yes,1-Up Mushroom,333,84,6.00,Super Mario Bros.,mario_data_20250101.csv,2025-10-28 15:21:26.079051
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,...,2.000000,Yes,No,"Red Shell, Banana Peel, Fire Flower",461,55,7.00,Super Mario World,mario_data_20250101.csv,2025-10-28 15:21:26.079051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,Toadette,Green Caps,Star World,Mach 8,Luma,B,A,S,8,Drifter,...,0.000000,No,Yes,"1-Up Mushroom, Super Mushroom",172,43,5.00,Super Mario Bros.,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47996,Bowser Jr.,Dino Buddies,Donut Plains,Circuit Special,Koopa Troopa,C,C,B,13,Speedster,...,1.083565,No,Unkown,Fire Flower,11,72,9.00,Super Mario World,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47997,Luigi,Green Caps,dONUT pLAINS,Biddybuggy,Shy Guy,A,D,D,27,Drifter,...,4.000000,Yes,Unkown,Super Star,460,25,2.00,Super Mario Odyssey,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47998,Toadette,Koopa Clan,mUSHROOM kINGDOM,Circuit Special,Yoshi,B,A,B,8,Item Specialist,...,0.000000,Yes,Yes,"Super Star, Banana Peel, Green Shell",82,39,4.00,Mario Kart 8 Deluxe,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040


In [ ]:
# making a dict of expected format/dtype for each column in dataframe
expected_formats ={
	'Player Name': 'string',
	'Team': 'string',
	'Kart Racing Rank': 'string',
	'Mushroom Cup Participation': 'bool',
	'Participation in Battle Mode': 'bool',
	'Platforming Rank': 'string',
	'Boss Battle Rank': 'string',
	'Power-Ups Owned': 'string',
	'Power-Ups Used': 'int32',
    'Kart Role':'string',
	'Team Points': 'int32',
	'Levels Completed': 'int32',
	'Lives Lost': 'int32',
	'Times Hit by Enemies': 'int32',
	'Vehicle Type': 'string',
	'World': 'string',
	'Coins Spent in Toad Town': 'int32',
	'Companion': 'string',
	'Primary Game': 'string',
    'filename':'string',
	'loadDateTimeStamp':'string'
}

In [ ]:
# checking if the columns are in the expected format
check_formats(df,expected_formats=expected_formats)

Below are incorrect formats
__________________________________________________
Correct column count: 0


,Columns,Actual,Expected
0,Player Name,object,string
1,Team,object,string
2,World,object,string
3,Vehicle Type,object,string
4,Companion,object,string
5,Kart Racing Rank,object,string
6,Platforming Rank,object,string
7,Boss Battle Rank,object,string
8,Power-Ups Used,int64,int32
9,Kart Role,object,string


In [ ]:
# converting oclumns to expected formats
convert_column_types(df,column_types=expected_formats)

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,...,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,filename,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,...,0,True,True,1-Up Mushroom,64,26,4,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,...,4,True,True,"Red Shell, Super Star",335,40,5,Mario Tennis Aces,mario_data_20250101.csv,2025-10-28 15:21:26.079051
2,Waluigi,Unknown Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,...,1,True,True,Green Shell,182,57,5,Mario Kart 8 Deluxe,mario_data_20250101.csv,2025-10-28 15:21:26.079051
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,...,5,True,True,1-Up Mushroom,333,84,6,Super Mario Bros.,mario_data_20250101.csv,2025-10-28 15:21:26.079051
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,...,2,True,True,"Red Shell, Banana Peel, Fire Flower",461,55,7,Super Mario World,mario_data_20250101.csv,2025-10-28 15:21:26.079051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,Toadette,Green Caps,Star World,Mach 8,Luma,B,A,S,8,Drifter,...,0,True,True,"1-Up Mushroom, Super Mushroom",172,43,5,Super Mario Bros.,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47996,Bowser Jr.,Dino Buddies,Donut Plains,Circuit Special,Koopa Troopa,C,C,B,13,Speedster,...,1,True,True,Fire Flower,11,72,9,Super Mario World,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47997,Luigi,Green Caps,dONUT pLAINS,Biddybuggy,Shy Guy,A,D,D,27,Drifter,...,4,True,True,Super Star,460,25,2,Super Mario Odyssey,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040
47998,Toadette,Koopa Clan,mUSHROOM kINGDOM,Circuit Special,Yoshi,B,A,B,8,Item Specialist,...,0,True,True,"Super Star, Banana Peel, Green Shell",82,39,4,Mario Kart 8 Deluxe,mario_data_20250601.jsonl,2025-10-28 15:21:27.251040


In [96]:
df['Player Name'].dtype

string[python]

In [ ]:
# checking mispellings in Player Name
check_mispelling(df,'Player Name',0.8)

,name1,name2,similarity
0,Lduigi,LuigiL,0.8000
1,Lduigi,Luiggi,0.8000
2,Lduigi,LuigLi,0.8000
3,Lduigi,Ludigi,1.0000
4,Lduigi,Luuigi,0.8000
...,...,...,...
21346,Bowser Jr.P,Bowser Jrg.,0.8182
21347,Dtaisy,Daiasy,0.8333
21348,Dalisy,Daiasy,0.8333
21349,Deaisy,Daiasy,0.8333


In [ ]:
# making a list of right names in Player Name
rename_values=[
"Mario","Luigi","Peach","Daisy","Yoshi","Toad","Toadette",
"Rosalina","Wario","Waluigi","Bowser","Bowser Jr."
]

In [ ]:
for r in rename_values:
        mask = df['Player Name'].apply(lambda x: check_similarity(r, x) >= 0.8)
        df.loc[mask, 'Player Name'] = r
# renaming values using a boolean mask within the df.loc[] 

check_mispelling(df,'Player Name',0.8)

,name1,name2,similarity


In [100]:
df['Player Name']

0             Yoshi
1             Peach
2           Waluigi
3             Yoshi
4        Bowser Jr.
            ...    
47995      Toadette
47996    Bowser Jr.
47997         Luigi
47998      Toadette
47999      Rosalina
Name: Player Name, Length: 48000, dtype: string

In [ ]:
# Validating for any mispellings
for r in df['Player Name']:
    if r not in rename_values:
        print(r)

Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Player
Unknown Pl

In [102]:
df['Player Name']

0             Yoshi
1             Peach
2           Waluigi
3             Yoshi
4        Bowser Jr.
            ...    
47995      Toadette
47996    Bowser Jr.
47997         Luigi
47998      Toadette
47999      Rosalina
Name: Player Name, Length: 48000, dtype: string

In [103]:
# cleaning trailing and leading white spaces
cols = ['Vehicle Type', 'World', 'Primary Game']
df[cols] = df[cols].apply(lambda x: x.str.strip())


In [104]:
#validating that no leading or trailing white space exists
for r in df['Primary Game']:
    if r.startswith(" ") or r.endswith(" "):
        print("Space exists")

In [105]:
for r in df['Vehicle Type']:
    if r.startswith(" ") or r.endswith(" "):
        print("Space exists")

In [106]:
for r in df['World']:
    if r.startswith(" ") or r.endswith(" "):
        print("Space exists")

In [107]:
#Capitalizing values
cols = ['Team', 'Companion', 'World']
df[cols] = df[cols].apply(lambda x: x.str.title())

In [108]:
# Validating capitalized values
df[['Team','Companion','World']]

,Team,Companion,World
0,Toad Brigade,Polterpup,Yoshi'S Island
1,Green Caps,Koopa Troopa,Donut Plains
2,Unknown Team,Goomba,Yoshi'S Island
3,Toad Brigade,Goomba,Star World
4,Koopa Clan,Toad,Mushroom Kingdom
...,...,...,...
47995,Green Caps,Luma,Star World
47996,Dino Buddies,Koopa Troopa,Donut Plains
47997,Green Caps,Shy Guy,Donut Plains
47998,Koopa Clan,Yoshi,Mushroom Kingdom


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Player Name                   48000 non-null  string
 1   Team                          48000 non-null  string
 2   World                         48000 non-null  string
 3   Vehicle Type                  48000 non-null  string
 4   Companion                     48000 non-null  string
 5   Kart Racing Rank              48000 non-null  string
 6   Platforming Rank              48000 non-null  string
 7   Boss Battle Rank              48000 non-null  string
 8   Power-Ups Used                48000 non-null  int32 
 9   Kart Role                     48000 non-null  string
 10  Team Points                   48000 non-null  int32 
 11  Lives Lost                    48000 non-null  int32 
 12  Participation in Battle Mode  48000 non-null  bool  
 13  Mushroom Cup Par

In [ ]:
# validating formats of columns
check_formats(df,expected_formats=expected_formats)

Validation Complete, no discrepancies


Saving as parquet file to preserve metadata

In [111]:
df.to_parquet('./Silver layer/Silver.parquet',index=False)